### AJUSTANDO REDES DL PARA NLP DE CLASIFICACION

Descarga la información de reviews de IMDB tal y como hemos hecho en clase.

In [1]:
import io
import numpy as np
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, TextVectorization

In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

In [ ]:
train_dir = "../aclImdb/train"
shutil.rmtree(dataset_dir + "/unsup")

Construye los datasets de train, validation y test igual que en clase

In [12]:
batch_size = 1024
seed = 123


train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation', # Esto y la semilla permiten que las muestras con train no se superpongan
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [13]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Construye la capa de vectorización para que ahora extraiga ella el tamaño de la secuencia a partir del dataset de entrada.
Adaptala al dataset de entrada, recuerda que tienes que crearte otro que no tenga el target incluido (en el notebook de clase tienes cómo hacerlo con una función lambda)

In [14]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data) # lo convierte a mayúsculas
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ') # le quita los codigos de cambio de línea
    return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '') #le quita los signos de puntuación


vocab_size = 10000 


vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int')
    #output_sequence_length=sequence_length)

text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Construye el modelo sin recurrentes que vimos en clase, pero haz que el número de neuronas de la capa densa que hace la regresión dependa de una variable (por ejemplo num_dense_neurons). Inicializa esta variable igualandola al número de dimensiones del embedding. 

In [15]:
embedding_dim=16
tune_parameter = 1
num_dense_neurons = embedding_dim * tune_parameter


model = Sequential([
  vectorize_layer, # 100 [1, 3, 4, 4, 90, ...]
  Embedding(vocab_size, embedding_dim, name="embedding"), # 10.000 x 16 --> [[], [], [] ...] 100x16
  GlobalAveragePooling1D(), # [] 16
  Dense(num_dense_neurons, activation='relu'), # 
  Dense(1) # originalmente no tiene activacion
])


Compila y entrena con la misma configuración de dimensiones de embedding y tamaño de vocabulario que en clase (10000), y el mismo número de épocas (15). Pista: Es posible que el tamaño de la secuencia le afecte, si es así tendrás que añadir algo a la definición del modelo y volver a compilar y entrenar

In [16]:
model.compile(optimizer='adam',
              # binary_crossentropy
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [17]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15)

Epoch 1/15
20/20 [==============================] - 15s 742ms/step - loss: 0.6930 - accuracy: 0.5028 - val_loss: 0.6930 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 5s 276ms/step - loss: 0.6927 - accuracy: 0.5028 - val_loss: 0.6925 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 5s 268ms/step - loss: 0.6920 - accuracy: 0.5028 - val_loss: 0.6913 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 5s 264ms/step - loss: 0.6909 - accuracy: 0.5028 - val_loss: 0.6897 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 5s 263ms/step - loss: 0.6891 - accuracy: 0.5028 - val_loss: 0.6872 - val_accuracy: 0.4886
Epoch 6/15
20/20 [==============================] - 6s 281ms/step - loss: 0.6865 - accuracy: 0.5028 - val_loss: 0.6835 - val_accuracy: 0.4886
Epoch 7/15
20/20 [==============================] - 6s 291ms/step - loss: 0.6825 - accuracy: 0.5028 - val_loss: 0.6779 - val_accuracy: 0.4886
Epoch

In [18]:

model = Sequential([
  vectorize_layer, # 100 [1, 3, 4, 4, 90, ...]
  Embedding(vocab_size, embedding_dim, name="embedding", mask_zero = True), # 10.000 x 16 --> [[], [], [] ...] 100x16
  GlobalAveragePooling1D(), # [] 16
  Dense(num_dense_neurons, activation='relu'), # 
  Dense(1) # originalmente no tiene activacion
])

In [19]:
model.compile(optimizer='adam',
              # binary_crossentropy
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [20]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15)

Epoch 1/15
20/20 [==============================] - 7s 301ms/step - loss: 0.6917 - accuracy: 0.5028 - val_loss: 0.6895 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 6s 284ms/step - loss: 0.6862 - accuracy: 0.5028 - val_loss: 0.6825 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 6s 292ms/step - loss: 0.6768 - accuracy: 0.5028 - val_loss: 0.6704 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 6s 296ms/step - loss: 0.6607 - accuracy: 0.5028 - val_loss: 0.6515 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 6s 303ms/step - loss: 0.6375 - accuracy: 0.5030 - val_loss: 0.6257 - val_accuracy: 0.4894
Epoch 6/15
20/20 [==============================] - 6s 300ms/step - loss: 0.6072 - accuracy: 0.5252 - val_loss: 0.5937 - val_accuracy: 0.5506
Epoch 7/15
20/20 [==============================] - 6s 284ms/step - loss: 0.5710 - accuracy: 0.6169 - val_loss: 0.5573 - val_accuracy: 0.6316
Epoch 

Evalua el modelo

In [22]:
test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir.replace("train","test"),
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)


Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [23]:
model.evaluate(test_ds)

20/20 [==============================] - 32s 942ms/step - loss: 0.3595 - accuracy: 0.8339


[0.3594972789287567, 0.8339499831199646]

Ahora juega con las dimensiones de salida de la capa de embedding y con el número de neuronas de la capa densa (siempre como un factor de las dimensiones de emmbeding) y logra que el modelo tenga una accuracy superior al 85% en validación en test. NO JUEGUES CON LAS EPOCAS, todavía

In [29]:
embedding_dims = 16
tune_parameter = 1
num_dense_neurons = embedding_dim * tune_parameter
desired_acc = 0.85

end = False
for tune_parameters in range(1,2,4):
  for embeddings in [embedding_dim * i for i in [1,2,4]]:
      index = "Model_e_%d_neurons_%d" %(embeddings, tune_parameter * embeddings)
      print("Modelo %s" %(index))
      num_dense_neurons = embeddings * tune_parameters
      model = Sequential([
        vectorize_layer, # 100 [1, 3, 4, 4, 90, ...]
        Embedding(vocab_size, embeddings, name="embedding", mask_zero = True), # 10.000 x 16 --> [[], [], [] ...] 100x16
        GlobalAveragePooling1D(), # [] 16
        Dense(num_dense_neurons, activation='relu'), # 
        Dense(1) # originalmente no tiene activacion
      ])
      model.compile(optimizer='adam',
            # binary_crossentropy
            loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
            metrics=['accuracy'])
      model.fit( train_ds,
            validation_data=val_ds,
            epochs=15)            
      loss_test, acc_test = model.evaluate(test_ds) 
      print("Accuracy:",acc_test)   
      if acc_test >= desired_acc:
          opt_embedding_dims = embedding_dims 
          opt_num_neurons = opt_embedding_dims * tune_parameters
          end = True
          break
  if end:
     break



Modelo Model_e_16_neurons_16
Epoch 1/15
20/20 [==============================] - 7s 298ms/step - loss: 0.6923 - accuracy: 0.5028 - val_loss: 0.6911 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 6s 287ms/step - loss: 0.6886 - accuracy: 0.5028 - val_loss: 0.6863 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 6s 295ms/step - loss: 0.6821 - accuracy: 0.5028 - val_loss: 0.6783 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 6s 301ms/step - loss: 0.6712 - accuracy: 0.5028 - val_loss: 0.6647 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 6s 293ms/step - loss: 0.6538 - accuracy: 0.5028 - val_loss: 0.6449 - val_accuracy: 0.4886
Epoch 6/15
20/20 [==============================] - 6s 294ms/step - loss: 0.6297 - accuracy: 0.5028 - val_loss: 0.6189 - val_accuracy: 0.4886
Epoch 7/15
20/20 [==============================] - 6s 301ms/step - loss: 0.5996 - accuracy: 0.5102 - val_loss: 0.5876 

NameError: name 'opt_embedding_dim' is not defined

In [33]:
opt_num_neurons = opt_embedding_dims * tune_parameters

Aumenta el número de épocas hasta conseguir la mejor accuracy posible con la configuración de embeddings y neuronas del paso anterior. Hazlo en un solo entrenamiento (pista, tendrás que usar un callback)

In [35]:
early_stop = tf.keras.callbacks.EarlyStopping(patience = 10, monitor = "val_accuracy", restore_best_weights= True)
model = Sequential([
        vectorize_layer, # 100 [1, 3, 4, 4, 90, ...]
        Embedding(vocab_size, opt_embedding_dims, name="embedding"), # 10.000 x 16 --> [[], [], [] ...] 100x16
        GlobalAveragePooling1D(), # [] 16
        Dense(opt_num_neurons, activation='relu'), # 
        Dense(1) # originalmente no tiene activacion
      ])
model.compile(optimizer='adam',
      # binary_crossentropy
      loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
      metrics=['accuracy'])
model.fit( train_ds,
            validation_data=val_ds,
            epochs=1500,
            callbacks=[early_stop])  

Epoch 1/1500
20/20 [==============================] - 4s 160ms/step - loss: 0.6931 - accuracy: 0.5028 - val_loss: 0.6929 - val_accuracy: 0.4886
Epoch 2/1500
20/20 [==============================] - 3s 157ms/step - loss: 0.6923 - accuracy: 0.5028 - val_loss: 0.6919 - val_accuracy: 0.4886
Epoch 3/1500
20/20 [==============================] - 3s 154ms/step - loss: 0.6914 - accuracy: 0.5028 - val_loss: 0.6905 - val_accuracy: 0.4886
Epoch 4/1500
20/20 [==============================] - 3s 155ms/step - loss: 0.6900 - accuracy: 0.5028 - val_loss: 0.6887 - val_accuracy: 0.4886
Epoch 5/1500
20/20 [==============================] - 3s 155ms/step - loss: 0.6880 - accuracy: 0.5028 - val_loss: 0.6857 - val_accuracy: 0.4886
Epoch 6/1500
20/20 [==============================] - 3s 158ms/step - loss: 0.6849 - accuracy: 0.5028 - val_loss: 0.6814 - val_accuracy: 0.4886
Epoch 7/1500
20/20 [==============================] - 3s 156ms/step - loss: 0.6805 - accuracy: 0.5028 - val_loss: 0.6756 - val_accuracy:

Evalúalo contra test

In [36]:
model.evaluate(test_ds) 

20/20 [==============================] - 4s 139ms/step - loss: 0.3115 - accuracy: 0.8728


[0.31152093410491943, 0.8727999925613403]

### VOLUNTARIO
Haz el mismo ejercicio utilizando una red recurrente

SOL Breve: Cambiar la capa de Pooling y la densa por una GRU o una LSTM en todas las definiciones de modelos con número de celdas dependiente del embedding con un factor 4 de partida